# 혐오발언 탐지와 분류 딥러닝모델 개발


- 상단 메뉴의 **"런타임"** -> **"런타임 유형 변경"** -> **"하드웨어 가속기"**를 **"GPU"**로 설정
- 허깅페이스의 `jeanlee_kmhas_korean_hate_speech` 파일을 학습데이터로 사용: https://huggingface.co/datasets/jeanlee/kmhas_korean_hate_speech
- 사전학습된 BERT 모델로 `beomi/KcELECTRA-base` 활용: https://huggingface.co/beomi/KcELECTRA-base
- 9개 레이블에 대한 멀티레이블 분류

In [ ]:
# GPU가 정상적으로 연결되었는지 확인 (Colab에서 실행 시)
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 1. 라이브러리 설치 및 임포트
 - `openpyxl`은 엑셀(xlsx) 파일을 읽기 위함
 - `sklearn`은 평가지표 계산에 필요
 - `transformers`는 Hugging Face 모델을 사용하기 위해 필수
 - `datasets`는 Trainer와 함께 사용할 수 있지만, 여기서는 직접 커스텀 Dataset 클래스를 만듦.

In [ ]:
!pip install transformers datasets sklearn openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

## 2. 데이터 불러와 학습데이터 분류


In [ ]:
# jeanlee_kmhas_korean_hate_speech.xlsx 파일을 Colab 환경에 업로드했다고 가정
# 파일 이름이나 경로가 다를 경우, 아래 부분을 수정해주세요.
df = pd.read_excel("/content/drive/MyDrive/파이썬특강/문과코딩_021725/jeanlee_kmhas_korean_hate_speech.xlsx")

print("데이터셋 예시 (상위 5개 행):")
display(df.head())

# 텍스트와 라벨 분리
label_names = ["Origin", "Physical", "Politics", "Profanity", "Age", "Gender", "Race", "Religion", "NotHate"]
X = df["text"].values  # 텍스트 컬럼
y = df[label_names].values  # 9개 레이블

# train, validation, test 8:1:1 분할
# 1) 먼저 10%를 test로 분할 -> (train+val)=90%, test=10%
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# 2) 남은 90% 중 1/9(약 11.11%)를 val로 분할 -> train=80%, val=10%, test=10%
val_ratio_of_temp = 1/9  # 0.1111...
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=val_ratio_of_temp, random_state=42)

print("\n데이터 분할 결과:")
print(f"Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")


## 3. 모델 불러오고 학습데이터 구성


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

# 사전학습된 모델 이름 (예: klue/bert-base)
model_name = "beomi/KcELECTRA-base"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Hugging Face Trainer를 사용하기 위해 list of dict 형태로 변환
train_data = [{"text": t, "labels": l} for t, l in zip(X_train, y_train)]
val_data = [{"text": t, "labels": l} for t, l in zip(X_val, y_val)]
test_data = [{"text": t, "labels": l} for t, l in zip(X_test, y_test)]

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        item = self.data[idx]
        text = item["text"]
        labels = item["labels"]
        # 토큰화 (return_tensors="pt" 없이 리스트 형태로 반환)
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding=False,
            max_length=128,
        )
        # 라벨을 float로 변환 (리스트 내 각 원소)
        encoding["labels"] = [float(x) for x in labels]
        return encoding

# Dataset 객체 생성
train_dataset = CustomDataset(train_data, tokenizer)
val_dataset = CustomDataset(val_data, tokenizer)
test_dataset = CustomDataset(test_data, tokenizer)

# 모델 로드 (num_labels=9, 멀티레이블)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_names),
    problem_type="multi_label_classification"
)

## 4. 평가지표 설정


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred):
    # eval_pred: (logits, labels)
    logits, labels = eval_pred
    # 시그모이드 적용
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.tensor(logits))
    # 0.5 기준으로 라벨 결정
    predictions = (probs > 0.5).int().numpy()
    labels = labels.astype(int)

    # 전체(멀티레이블) micro/macro F1
    f1_micro = f1_score(labels, predictions, average='micro', zero_division=0)
    f1_macro = f1_score(labels, predictions, average='macro', zero_division=0)

    # 멀티레이블에서 accuracy_score는 각 샘플의 모든 레이블이 일치해야 1
    overall_accuracy = accuracy_score(labels, predictions)

    # 레이블별 accuracy, f1 계산
    results = {
        "overall_accuracy": overall_accuracy,
        "f1_micro": f1_micro,
        "f1_macro": f1_macro
    }

    for i, label_name in enumerate(label_names):
        # 각 레이블에 대해 accuracy, f1
        label_acc = accuracy_score(labels[:, i], predictions[:, i])
        label_f1 = f1_score(labels[:, i], predictions[:, i], zero_division=0)
        # dict에 저장
        results[f"{label_name}_acc"] = label_acc
        results[f"{label_name}_f1"] = label_f1

    return results

## 5. TrainingArguments 설정 및 Trainer 학습


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",          # 모델 및 로그가 저장될 폴더
    evaluation_strategy="epoch",     # 매 epoch마다 평가
    save_strategy="epoch",          # 매 epoch마다 체크포인트 저장
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,               # 예시로 3 epoch
    weight_decay=0.01,               # AdamW의 weight decay
    logging_dir="./logs",           # 로그 디렉토리
    logging_steps=50,                # 로깅 빈도
    load_best_model_at_end=True      # 최적 모델 로드
)

# DataCollatorWithPadding: 동적 패딩 적용
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# 학습 진행
trainer.train()

# 검증 데이터 성능 확인
val_result = trainer.evaluate()
print("\n[Validation Metrics]")
for k, v in val_result.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}")
    else:
        print(f"{k}: {v}")

## 6. 테스트 세트 평가와 저장


In [ ]:
# test 데이터 평가
test_result = trainer.evaluate(test_dataset)

print("\n[Test Metrics]")
for k, v in test_result.items():
    if isinstance(v, float):
        print(f"{k}: {v:.4f}")
    else:
        print(f"{k}: {v}")

In [ ]:
mymodel_path="/content/drive/MyDrive/파이썬특강/문과코딩_021725/혐오발언모델_022125"  # 폴더 지정
trainer.save_model(mymodel_path)

## 7. 저장된 모델 불러오기 및 Inference 예시


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# 1) 모델 & 토크나이저 로드
mymodel_path="/content/drive/MyDrive/파이썬특강/문과코딩_021725/혐오발언모델_022125"
model = AutoModelForSequenceClassification.from_pretrained(mymodel_path)
tokenizer = AutoTokenizer.from_pretrained(mymodel_path)
model.eval()

# 3) 추론(Inference) 함수 예시
def predict(text_list, model, tokenizer, threshold=0.5):
    # text_list: 문자열 또는 문자열 리스트
    if isinstance(text_list, str):
        text_list = [text_list]
    # 토큰화
    inputs = tokenizer(
        text_list,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=128
    )
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.sigmoid(logits)
        preds = (probs > threshold).int().cpu().numpy()
    return preds, probs.cpu().numpy()

# 4) 예시 문장
example_texts = [
    "빨갱이 집단이 판치는 세상이야.",
    "비천한 출신과 어울리려니까 힘드네.",
    "꼰대 아저씨들 빨리 은퇴시켜라",
    "특정 종교나 인종을 비하하는 내용이 담겼나요?"
]

predictions, probabilities = predict(example_texts, model, tokenizer)

# 결과 출력
for text, pred_vec, prob_vec in zip(example_texts, predictions, probabilities):
    print("\n문장:", text)
    print("모델 예측(0/1):", pred_vec)
    print("확률:", [round(p, 3) for p in prob_vec])
    # 레이블명을 함께 출력해볼 수도 있음
    for label_name, val in zip(label_names, pred_vec):
        print(f"  - {label_name}: {val}")

# 결과를 DataFrame에 저장
data = []
for text, pred_vec, prob_vec in zip(example_texts, predictions, probabilities):
    # 각 문장마다 dictionary 생성 (문장, 그리고 각 레이블별 예측과 확률)
    result = {"문장": text}
    for label, pred, prob in zip(label_names, pred_vec, prob_vec):
        result[f"{label}_pred"] = pred
        result[f"{label}_prob"] = round(prob, 3)
    data.append(result)

df_result = pd.DataFrame(data)
print(df_result)